# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [ ]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [ ]:
A_states = set(range(3))
A_actions = set(range(1))

print(f'{A_states=} | {A_actions=}')

T_A = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}
print(f'Transition function {T_A=}')

A_mdp = A_states, A_actions, T_A

A_states={0, 1, 2} | A_actions={0}
Transition function T_A={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [ ]:
A_mtx = np.zeros((3,3))
A_vals = np.array(list(T_A.values()))
A_mtx[np.arange(3), A_vals] = 1
A_mtx

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [ ]:
N = 2
An = np.linalg.matrix_power(A_mtx, N)
print(An[0,1])
N = 1000
An = np.linalg.matrix_power(A_mtx, N)
print(An[0,1])

0.0
0.0


Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [ ]:
B_states = set(range(4))
B_actions = set(range(3))

print(f'{B_states=} | {B_actions=}')

T_B = {
    (0, 0): 1,
    (0, 1): 2,
    (0, 2): 3
}
print(f'Transition function {T_B=}')

B_mdp = B_states, B_actions, T_B

B_states={0, 1, 2, 3} | B_actions={0, 1, 2}
Transition function T_B={(0, 0): 1, (0, 1): 2, (0, 2): 3}


In [ ]:
B_mtx = np.eye(4)
B_mtx[0,0]=0
B_vals = np.array(list(T_B.values()))
B_mtx[0, B_vals] = 1/3
B_mtx

array([[0.        , 0.33333333, 0.33333333, 0.33333333],
       [0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [ ]:
N = 2
Bn = np.linalg.matrix_power(B_mtx, N)
print(Bn[0,1])
N = 1000
Bn = np.linalg.matrix_power(B_mtx, N)
print(Bn[0,1])

0.3333333333333333
0.3333333333333333


In [ ]:
C_states = set(range(4))
C_actions = set(range(2))

print(f'{C_states=} | {C_actions=}')

T_C = {
    (0, 0): 1,
    (0, 1): 2,
    (1, 0): 1,
    (1, 1): 3,
    (2, 0): 3,
    (2, 1): 2,
}
print(f'Transition function {T_C=}')

C_mdp = C_states, C_actions, T_C


C_states={0, 1, 2, 3} | C_actions={0, 1}
Transition function T_C={(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2}


In [ ]:
C_mtx = np.array([[0, 0.5, 0.5, 0],
                 [0, 0.5, 0, 0.5],
                 [0, 0, 0.5, 0.5],
                 [0, 0, 0, 1]])

In [ ]:
N = 2
Cn = np.linalg.matrix_power(C_mtx, N)
print(Cn[0,1])
N = 1000
Cn = np.linalg.matrix_power(C_mtx, N)
print(Bn[0,1])

0.25
0.3333333333333333


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [ ]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

В качестве дополнения, запишите стратегию агента

In [ ]:
class Oracle:
  def __init__(self, R):
    self.R = R
  def act(self, state):
    best = -np.inf
    action = None
    for k,v in self.R.items():
      if k[0] == state:
        if v > best:
          best = v
          action = k[1]
    return action




## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [ ]:
A_R = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}
A_terminal = {2}
print(A_R)

A_mdp = *A_mdp, A_R
print(A_mdp)

{(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0})


In [ ]:
B_R = {}
B_terminal = {1, 2, 3}
for i in B_states:
  for j in B_actions:
    B_R[(i,j)] = np.random.rand()
for el in terminal:
  for j in B_actions:
    B_R[(i,j)] = 0
B_mdp = *B_mdp, B_R
B_R

{(0, 0): 0.16267142477541396,
 (0, 1): 0.1473654946226972,
 (0, 2): 0.7423486725078765,
 (1, 0): 0.437366814854246,
 (1, 1): 0.0011258699662193283,
 (1, 2): 0.8689692617523015,
 (2, 0): 0.6467428244595352,
 (2, 1): 0.4996095742476935,
 (2, 2): 0.5230699661263218,
 (3, 0): 0,
 (3, 1): 0,
 (3, 2): 0}

In [ ]:
C_R = {}
C_terminal = {3}
for i in C_states:
  for j in C_actions:
    C_R[(i,j)] = np.random.rand()
for el in terminal:
  for j in C_actions:
    C_R[(i,j)] = 0
C_mdp = *C_mdp, C_R
C_R

{(0, 0): 0.9223614885921632,
 (0, 1): 0.10866944901848874,
 (1, 0): 0.20208980760882989,
 (1, 1): 0.43005895623766344,
 (2, 0): 0.8520765336094754,
 (2, 1): 0.0007898708196252269,
 (3, 0): 0,
 (3, 1): 0}

## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [ ]:
def run_episode(mdp, agent, terminal, n_steps=10):
    states, actions, T, R = mdp
    s = 0
    tau = []
    for _ in range(n_steps):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next
        if s in terminal:
          break

    return tau

print(run_episode(A_mdp,Agent(A_actions), A_terminal))
print(run_episode(A_mdp,Oracle(A_R), A_terminal))

[(0, np.int64(0), -0.1), (1, np.int64(0), 1.0)]
[(0, 0, -0.1), (1, 0, 1.0)]


In [ ]:
print(run_episode(B_mdp,Agent(B_actions), B_terminal))
print(run_episode(B_mdp,Oracle(B_R), B_terminal))

[(0, np.int64(1), 0.1473654946226972)]
[(0, 2, 0.7423486725078765)]


In [ ]:
print(run_episode(C_mdp,Agent(C_actions), C_terminal))
print(run_episode(C_mdp,Agent(C_actions), C_terminal))
print(run_episode(C_mdp,Oracle(C_R), C_terminal))

[(0, np.int64(0), 0.9223614885921632), (1, np.int64(1), 0.43005895623766344)]
[(0, np.int64(1), 0.10866944901848874), (2, np.int64(0), 0.8520765336094754)]
[(0, 0, 0.9223614885921632), (1, 1, 0.43005895623766344)]


Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [ ]:
def get_G(trajectory):
  sum = 0
  for step in trajectory:
    _, _, r = step
    sum += r
  return sum

def get_mean_G(mdp, terminal, agent, n_trajectories=10):
  sum = 0
  for i in range(n_trajectories):
    trajectory = run_episode(mdp,agent,terminal)
    sum += get_G(trajectory)
  return sum / n_trajectories


In [ ]:
get_mean_G(A_mdp, A_terminal, Agent(A_actions))

0.9000000000000001

In [ ]:
get_mean_G(B_mdp, B_terminal, Agent(B_actions))

0.21145559145703013

In [ ]:
get_mean_G(C_mdp, C_terminal, Agent(C_actions))

1.3771629258981959